# Summer Clothes Sales EDA

1. Data source: https://www.kaggle.com/jmmvutu/summer-products-and-sales-in-ecommerce-wish
2. Data description
    1. The data comes from the e-commerce platform, Wish.
    2. The products listed in the dataset are the ones that would appear if you type "summer" in the search field of the platform. The data was scraped in the french localisation (hence some non-ascii latin characters such as « é » and « à ») in the title column.
    3. This dataset contains product listings as well as products ratings and sales performance, which you would not find in other datasets.
3. EDA goals:
    1. to identify the features that significantly affect the summer clothes sales.
    2. to practice basic Python coding.
    3. to take baby steps to data visualization.
    4. to interpret EDA results to obtain insights that lead to further statistical analysis.
4. Table of Contents
    1. Data Exploration
    2. Cleaning Dataset
    3. Graphical Analysis

### 1. Data Exploration

In [ ]:
# Import libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected = True)

In [ ]:
# Import original data (csv file)
summer = pd.read_csv('../input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')

# Print top five observations
summer.head()

In [ ]:
# Data information
summer.info()

In [ ]:
# Check null values, data type, and unique values in each column

null = summer.isnull().sum().to_frame(name='nulls').T
dtype = summer.dtypes.to_frame(name='dtypes').T
nunique = summer.nunique().to_frame(name='unique').T
pd.concat([null, dtype, nunique], axis=0)

In [ ]:
# Statistic summary for continuous variables
summer.describe()

### 2. Cleaning Dataset
#### 1) Missing value treatments

In [ ]:
# Detect the total number of missing values in each column
summer_nullity = summer.isnull().sum()
print(summer_nullity)

In [ ]:
# Visualize missing values 
# (1): as a matrix
msno.matrix(summer)
plt.show()

# (2): as a bar
msno.bar(summer)
plt.show()

Looking at the missing value matrix, there seems to be correlation among rating_#_count and between has_urgency_banner and urgency_text. With further examinations, it will be easier to determine which variables having missing values should stay or be dropped.

In [ ]:
# Missing patterns of dataset sorted by 'rating_#_count'
rating_sorted = summer.sort_values('rating_five_count')
msno.matrix(rating_sorted)
plt.show()

It seems that all missing values in each rating_#_count are Missing not at Random (MNAR), meaning there is relationship between missingness and its value, missing or non-missing.

Even though rating_count is the sum of values of each rating_#_count, the rating_count columns that have missing values in each rating_#_count do not have any missing values as total. So, it is necessary to replace these values in rating_count column with 0.

In [ ]:
# Missing Patterns of the dataset sorted by 'has_urgency_banner'
urgency_sorted = summer.sort_values('has_urgency_banner')
msno.matrix(urgency_sorted)
plt.show()

It seems that all missing values in has_urgency_banner and urgency_text are also MNAR. It is necessary to replace all these missing values with 0.

In [ ]:
# Visualize the correlation between the number of missing values in different columns 
# (1) as a heatmap 
msno.heatmap(summer)
plt.show()

#(2) as a dendrogram
msno.dendrogram(summer)
plt.show()

In the heatmap, a value near 1 means if one variable appears then the other variable is very likely to be present, meaning that if one variable appears in one of the rating_#_count columns, the other variable is highly likely to be present in the other rating_#_count.

The same interpretation applies to those two columns, has_urgency_banner and urgency_text.

In [ ]:
# Subset multiple columns
summer[["has_urgency_banner", "urgency_text"]]

In [ ]:
# Count the values of the column has_urgency_banner
summer["has_urgency_banner"].value_counts()

In [ ]:
# Count the values of the column urgency_text
summer["urgency_text"].value_counts()

In [ ]:
# Check if the missing values in both columns match
print(summer.loc[summer["has_urgency_banner"].isna(), "urgency_text"].unique())

# Check if the non-missing values in both columns match
print(summer.loc[~summer["has_urgency_banner"].isna(), "urgency_text"].unique())

It seems that the value of 1 in has_urgency_banner means a product has an urgent text on it, which is either 'Limited Quantity!' or 'Discount on wholesale purchases'. Since both columns are higly related and convey the same information, it is desirable to delete the column, urgency_text.

In [ ]:
# Removing the column urgency_text
summer.drop(["urgency_text"], axis=1, inplace=True)

# Data imputation
# Re-defining the column has_urgency_banner and fill the missig values with 0
summer["has_urgency_banner"] = summer["has_urgency_banner"].fillna(0)

# Confirm changes made
summer["has_urgency_banner"]

In [ ]:
# Group columns related to rating and define as rating_cols
rating_cols = ["rating", "rating_count", "rating_five_count", "rating_four_count", 
               "rating_three_count", "rating_two_count", "rating_one_count"]

# Define no_vote as the columns that have missing values
no_vote = summer.loc[summer[rating_cols].isna().any(axis=1), rating_cols]

# Set the value of 0 to each section that has missing values 
summer.loc[no_vote.index, rating_cols] = 0

# Print top five obs
summer.loc[no_vote.index, rating_cols].head()

Now all rows that once had missing values in each rating_#_count column have the value of 0 in every column. 

#### 2) Variable selection

In [ ]:
# Remove unimportant variables or variables that have a few missing values
drop_cols = ["title", "currency_buyer", "tags", "product_color", "product_variation_size_id", 
             "product_variation_inventory", "shipping_option_name", "shipping_is_express", 
             "countries_shipped_to", "inventory_total", "origin_country", "merchant_title", 
             "merchant_name", "merchant_info_subtitle", "merchant_id", "merchant_profile_picture",
             "product_url", "product_picture", "product_id", "theme", "crawl_month"]

summer.drop(drop_cols, axis=1, inplace=True)
print(summer.columns)

In [ ]:
# Remove duplicate rows - if any
summer.drop_duplicates(keep='first', inplace=True)

### 3. Graphical Analysis

The main goal here is to carry out a graphical univariate or multivariate analysis between major predictor variables and bewteen response variable; units_sold and other predictor variables.

#### 1) Pricing: price, retail_price, shipping_option_price

In [ ]:
# Histogram for price

sns.set()
plt.figure(figsize=(8, 6))
plt.hist(summer['price'], bins=20, facecolor='navy', range=[0, 50])
plt.xlabel("price (€)", fontsize=15)
plt.ylabel("number of unit sold", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title("Price Distribution", fontsize=17)
plt.show()

In [ ]:
# Histogram for retail price

plt.subplot(1, 2, 1)
(summer['retail_price']).plot.hist(bins=50, figsize=(12, 6), facecolor='gray', range=[0, 175])
plt.xlabel("retail price (€)", fontsize=15)
plt.ylabel("number of unit sold", fontsize=15)
plt.title("Retail Price Distribution", fontsize=17)
plt.xticks(fontsize=13)
plt.xticks(fontsize=13)
plt.show()

plt.subplot(1, 2, 2)
np.log(summer['retail_price']+1).plot.hist(bins=50, figsize=(12, 6), facecolor='gray')
plt.xlabel("log(retail price + 1)", fontsize=15)
plt.ylabel("number of unit sold", fontsize=15)
plt.title("Retail Price Distribution", fontsize=17)
plt.xticks(fontsize=13)
plt.xticks(fontsize=13)
plt.show()

In [ ]:
# Kernel Density Estimates (KDE) for both prices

plt.figure(figsize=(8, 6))
sns.kdeplot(summer['price'], shade=True)
sns.kdeplot(summer['retail_price'], shade=True)
plt.xlabel('price (€)', fontsize=15)
plt.ylabel('probability of density', fontsize=15)
plt.title("Price vs Retail Price", fontsize=17)
plt.show()

In [ ]:
# Plot data using Empirical Cumulative Distribution Function (ECDF)
# Compute and define ECDF
def ecdf(data):
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n+1) / n
    return x, y

# Plot ECDFs and Comparison of ECDFs
x_pri, y_pri = ecdf(summer['price'])
x_re_pri, y_re_pri = ecdf(summer['retail_price'])

plt.figure(figsize=(8, 6))
plt.plot(x_pri, y_pri, marker='.', linestyle='none')
plt.plot(x_re_pri, y_re_pri, marker='.', linestyle='none', color='red')

plt.legend(('price', 'retail price'), loc='lower right')
plt.xlabel('price (€)', fontsize=15)
plt.ylabel('ECDF', fontsize=15)
plt.title("Price vs Retail Price (1)", fontsize=17)
plt.margins(0.02) # Keep data off plot edges
plt.show()

# Produce CDF using Seaborn
plt.figure(figsize=(8, 6))
sns.kdeplot(summer['price'], shade=True, cumulative=True)
sns.kdeplot(summer['retail_price'], shade=True, cumulative=True)
plt.xlabel('price (€)', fontsize=15)
plt.ylabel('CDF', fontsize=15)
plt.title("Price vs Retail Price (2)", fontsize=17)
plt.show()

In [ ]:
# Check the distribution of shipping_option_price

plt.figure(figsize=(8, 6))
sns.violinplot('shipping_option_price', data=summer, palette='muted', 
               scale='count', inner='quartile')
plt.xlabel('shipping price range (€)', fontsize=15)
plt.title('Shipping Price Distribution', fontsize=17)
plt.show()

In [ ]:
# Correlation between units_sold and price variables

plt.figure(figsize=(8, 6))
summer_price = summer[["units_sold", "price", "retail_price", "shipping_option_price"]]
sns.heatmap(summer_price.corr(), annot=True)
plt.title('Correlation between Units sold and Prices', fontsize=15)
plt.show()

Observations
1. The variable price is normally distributed and the majority of values range between 0 and 20.
2. The variable retail_price is severely right skewed and the majority of values range between 0 and 25, having  much larger variance compared to price.
3. From the KDE plot and ECDF, it is clear that about 30% of retail_price values widely range bewteen about 25 and 250 compared to 30% of price values ranging between about 12.5 and 25.
4. In the violinplot, the majority of values of shipping_option_price range between 1 and 3, which indicates potential correlation with product sales.
5. Surprisingly, none of price variables show strong correlation with units_sold. Rather, there is strong positive correlation between price and shipping price, 0.87, and weak positive correlation between price and retail price, 0.31.

Inspiration
1. Since the price is the actual price for customers to pay and the retail price is the reference price from other stores or websites, a product sells more as the difference between those two price gets bigger?
2. How do these prices relate to the number of unit sold?

#### 2) Categorical variables: uses_ad_boosts, badges_count, has_urgency_banner

In [ ]:
# Count the number of usage of ad boots

plt.figure(figsize=(8, 6))
sns.countplot('uses_ad_boosts', data=summer, palette="Set2")
plt.xlabel('Ad-boots Usage', fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.xticks([0, 1], ["No", "Yes"], fontsize=13)
plt.title('Ad-boots Usage', fontsize=17)
plt.show()

In [ ]:
# Correlation between units_sold and uses_ad_boosts

corr = summer['uses_ad_boosts'].corr(summer['units_sold'])
print(f"The Correlation coefficient between units_sold and uses_ad_boosts is: {np.round(corr, 4)}")

In [ ]:
# KDE plot for ad-boosts usage and unit sold

g = sns.FacetGrid(summer, hue='uses_ad_boosts', height=4, aspect=3)

g = (g.map(sns.kdeplot, 'units_sold', shade=True).add_legend())
max_units = summer['units_sold'].max()
g.set(xlim=(0, max_units))
plt.title('Ad-boosts Usage vs Unit Sold', fontsize=17)
plt.show()

Observations

1. Looking at the KDE plot, both lines almost overlap, meaning using ad boosts on a product does not significantly affect its sale. Whether a seller uses an ad boost or not, it does not lead to successful product sales.

In [ ]:
# Correlation between unit sold and badges

plt.figure(figsize=(8, 6))
badge_group = summer[["badges_count", "badge_local_product", 
                      "badge_product_quality", "badge_fast_shipping", "units_sold"]]
sns.heatmap(badge_group.corr(), annot=True)
plt.title('Correlation between Units sold and Badges', fontsize=15)
plt.show()

In [ ]:
# Association between units sold and badges on product

g = sns.catplot(x="badges_count", y="units_sold", data=summer, 
                kind='box', palette="Set3", aspect=2)
(g.set(ylim=(0, 60000))) 
plt.xlabel("total number of badges", fontsize=15)
plt.ylabel("unit sold", fontsize=15)
plt.title("Association between Unit sold and Badge", fontsize=17)
plt.show()

Observations

1. Unexpectedly, no correlation is found between product sales and all types of badges from the heatmap. The correlations found between badges_count and badge_local_product and between badges_count and badge_fast_shipping are not considered significant as badges_count is simply the sum of any badges a product has. 

2. This non-correlation can be found in the boxplots as well. The number of any badges a product has does not significantly affect the sale of product.

In [ ]:
# Association between urgency banner and unit sold
# Use pointplot

sns.catplot(x="has_urgency_banner", y="units_sold", data=summer, kind="point", joint=True)
plt.xlabel("urgency banner", fontsize=15)
plt.ylabel("unit sold", fontsize=15)
plt.xticks([0.0, 1.0], ["No", "Yes"], fontsize=13)
plt.title("Association between has_urgency_banner and units_sold (1)", fontsize=15)
plt.show()

In [ ]:
# Use violinplot

sns.catplot(x="has_urgency_banner", y='units_sold', data=summer, kind="violin", palette="Set1")
plt.xlabel("urgency banner", fontsize=15)
plt.ylabel("unit sold", fontsize=15)
plt.xticks([0.0, 1.0], ["No", "Yes"], fontsize=13)
plt.title("Association between has_urgency_banner and units_sold (2)", fontsize=15)
plt.show()

Observations

1. In the pointplot, a point in each line indicates the mean of unit sold in each category. For example, the mean of product sold that has no urgency banner is about 4600. 

2. The line that connects two points in each line allows interaction to be judged by its slope. There is an interaction between two levels. This interaction, however, just reveal an urgency banner has no effect on product sales. Because the product that has the urgency banner shows lower sales. 

3. Looking at the almost identical distribution of values at each level, it is confirmed again the urgency banner has no impact on product sales.

#### 3) Rating: rating, rating_count, rating_one to five_count

In [ ]:
# Define function for range categorization of units_sold

def num_units_sold(units_sold):
    units_sold = int(units_sold)
    
    bracket = ''
    if units_sold in range(0, 100):
        bracket = '< 100'
    if units_sold in range(100, 1000):
        bracket = '100 - 1000'
    if units_sold in range(1000, 5000):
        bracket = '1000 - 5000'
    if units_sold in range(5000, 10000):
        bracket = '5000 - 10000'
    if units_sold in range(10000, 20000):
        bracket = '10000 - 20000'
    if units_sold in range(20000, 50000):
        bracket = '20000 - 50000'
    if units_sold in range(50000, 100000):
        bracket = '> 50000'
    return bracket

In [ ]:
# Add up the all counts of each rating level in given range category

summer["num_units_sold"] = summer["units_sold"].apply(num_units_sold)
units_sold = summer['num_units_sold'].unique().tolist()
units_sold_groupby = summer.groupby("num_units_sold").agg({
    'rating_five_count': 'sum',
    'rating_four_count': 'sum',
    'rating_three_count': 'sum',
    'rating_two_count': 'sum',
    'rating_one_count': 'sum'})
units_sold_groupby.reset_index()
units_sold_groupby = units_sold_groupby.iloc[1:]
units_sold_groupby

In [ ]:
# Check association between rating_#_count and units_sold using barchart

unit_range = ["< 100", "100 - 1000", "1000 - 5000", "5000 - 10000", 
              "10000 - 20000", "20000 - 50000", "> 50000"]
rat_1 = ["47.0", "2264.0", "13459.0", "21971.0", "32677.0", "48738.0", "17576.0"]
rat_2 = ["40.0", "1187.0", "7769.0", "13327.0", "21399.0", "33191.0", "12960.0"]
rat_3 = ["58.0", "2328.0", "16041.0", "27271.0", "46207.0", "70335.0", "27439.0"]
rat_4 = ["89.0", "3151.0", "21729.0", "36015.0", "65087.0", "93245.0", "35783.0"]
rat_5 = ["225.0", "7529.0", "54169.0", "90094.0", "164130.0", "226350.0", "88685.0"]

layout = go.Layout(barmode="stack")

fig = go.Figure(data=[
    go.Bar(x=unit_range, y=rat_5, name="rating_five_count"),
    go.Bar(x=unit_range, y=rat_4, name="rating_four_count"),
    go.Bar(x=unit_range, y=rat_3, name="rating_three_count"),
    go.Bar(x=unit_range, y=rat_2, name="rating_two_count"),
    go.Bar(x=unit_range, y=rat_1, name="rating_one_count")], layout=layout)
iplot(fig)

In [ ]:
# define function for categorization of rating

def rating_category(rating):
    rating = int(rating)
    
    bracket = ''
    if rating in range (1, 3):
        bracket = "poor"
    if rating in range (3, 4):
        bracket = "fair"
    if rating in range (4, 6):
        bracket = "good"
    return bracket

In [ ]:
# Correlation between unit sold and ratings using scatterplot

summer["avg_rating"] = summer["rating"].apply(rating_category)

plt.figure(figsize=(8, 6))

sns.scatterplot(x='units_sold', y='rating_count', hue='avg_rating', 
                palette='Pastel1', size='avg_rating', sizes=(10, 200), data=summer)
plt.xlabel("number of units sold", fontsize=15)
plt.ylabel("total number of rating", fontsize=15)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000], ['0', '20k', '40k', '60k', '80k', '100k'], 
           fontsize=13)
plt.yticks([0, 5000, 10000, 20000], ['0', '5k', '10k', '20k'], fontsize=13)
plt.title("Correlation between unit sold and ratings", fontsize=17)
plt.show()

Observations

1. Looking at the barchart, there exists correlation between product sales and each rating level. As the range of the number of units sold goes up, the proportion of each rating level increases as well, meaning a product that has a good rating, more than rating 4, sells well. 
2. Noticeably, as a product sells well, a poor rating, less than rating 2, also increases accordingly even if its increase is smaller than that of other rating levels, such as rating 4 or 5.
3. It is assumable that the reason why the range of more than 50k in units sold has much fewer values compared to other range categories is that there might have not been rating evaluation given from customers. However, since the rating 4 and 5 take more than half of the chart, it can conclude that the higher the rating is, the better the product sells.
4. In the scatterplot, the tendency of the positive association can be observed, which means as a product that has a good rating, defined as more than rating 4, sells well and has a much larger total number of ratings.

#### 4) Merchant factors: merchant_rating_count, merchant_rating, merchant_has_profile_picture

In [ ]:
# Correlation coefficient between units_sold and merchant variables 

plt.figure(figsize=(8, 6))
merchant_group = summer[["merchant_rating_count", "merchant_rating", 
                      "merchant_has_profile_picture", "units_sold"]]
sns.heatmap(merchant_group.corr(), annot=True, cmap="YlGnBu")
plt.title('Correlation between units_sold and merchant variables', fontsize=15)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.show()

In [ ]:
# Correlation between units_sold and merchant_profile & rating using lineplot

summer["merchant_profile"] = summer["merchant_has_profile_picture"].rename().apply(lambda x: "Yes" if x==1 else "No")

plt.figure(figsize=(8, 6))

sns.lineplot(x="units_sold", y="merchant_rating", hue="merchant_profile", hue_order=['Yes', 'No'], 
             style="merchant_profile", palette="ocean_r", data=summer)
plt.xlabel("number of units sold", fontsize=15)
plt.ylabel("mean of merchant rating", fontsize=15)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000], ['0', '20k', '40k', '60k', '80k', '100k'], 
           fontsize=13)
plt.title("Correlation between units_sold and merchant_profile & rating", fontsize=17)
plt.show()

In [ ]:
# Define function for categorization of merchant_rating

def avg_mer_rating(merchant_rating):
    merchant_rating = int(merchant_rating)
    
    bracket = ''
    if merchant_rating in range (0, 3):
        bracket = "poor"
    if merchant_rating in range (3, 4):
        bracket = "fair"
    if merchant_rating in range (4, 6):
        bracket = 'good'
    return bracket

In [ ]:
# Correlation between units_sold and merchant rating

summer["avg_mer_rating"] = summer["merchant_rating"].apply(avg_mer_rating)
summer["merchant_profile"] = summer["merchant_has_profile_picture"].rename()

plt.figure(figsize=(8, 6))

sns.lineplot(x="units_sold", y="merchant_rating_count",  hue="avg_mer_rating", style="avg_mer_rating", 
             palette="magma_r", data=summer)
plt.xlabel("number of units sold", fontsize=15)
plt.ylabel("total number of merchant rating", fontsize=15)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000], ['0', '20k', '40k', '60k', '80k', '100k'], 
           fontsize=13)
plt.yticks([0, 50000, 100000, 150000, 200000, 250000, 300000, 350000], 
           ['0', '50k', '100k', '150k', '200k', '250k', '300k', '350k'], fontsize=13)
plt.title('Correlation between units_sold and merchant_rating & count', fontsize=17)
plt.show()

Observations

1. Let's look at the heatmap. There are quite weak but positive correlations between units_sold and other merchant variables. 
2. Looking at the first lineplot, interestingly, in the range of the number of units sold between about 35k and 65k, the products without a merchant profile have a higher mean of merchant rating than the products with a merchant profile. Except for this range, it can conclude that customers prefer a product with a merchant profile on, as it assumably provides them with some credibility of the merchant.
3. In the last lineplot, it is observable that there is positive correlation between units_sold and merchant_rating_count, meaning the more merchant ratings a product has, the better the product sells. In addition, a product with a good rating, more than 4 stars, tends to have much more merchant ratings as its sales go up.